# Modules import

In [0]:
# Standard modules
import os
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import boto3

# tensorflow modules
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# pyspark modules
import pyspark 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, split
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.feature import StringIndexer, StandardScaler
from pyspark.ml.feature import PCA

# Spark initialization

In [0]:
# Instanciation of spark with credientials

spark = (SparkSession.builder        
         .appName('jv-p8')
         .config('spark.hadoop.fs.s3a.access.key', <access key>)
         .config('spark.hadoop.fs.s3a.secret.key', <secret key>)
         .config('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') 
         .getOrCreate()
        )

sc = spark.sparkContext
sc.setSystemProperty('com.amazonaws.services.s3a.enableV4', 'true')
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.eu-west-1.amazonaws.com")

# Loading the data from the S3 bucket

In [0]:
# Vizualisation of the data

connection = boto3.client('s3')
contents = connection.list_objects(Bucket='s3-jv-ocr',
                                  Prefix='reduced_dataset/')['Contents']
for file in contents:
    print(file['Key'])

reduced_dataset/Banana/0_100.jpg
reduced_dataset/Banana/73_100.jpg
reduced_dataset/Cherry_1/0_100.jpg
reduced_dataset/Cherry_1/153_100.jpg
reduced_dataset/Eggplant/0_100.jpg
reduced_dataset/Eggplant/199_100.jpg
reduced_dataset/Kiwi/0_100.jpg
reduced_dataset/Kiwi/174_100.jpg

In [0]:
# Loading the data

s3_url = "s3a://s3-jv-ocr/reduced_dataset/*"
image_df = image_df = spark.read.format("binaryfile").load(s3_url)
image_df.printSchema()

root
-- path: string (nullable = true)
-- modificationTime: timestamp (nullable = true)
-- length: long (nullable = true)
-- content: binary (nullable = true)

In [0]:
# Data overview

image_df.show()

+--------------------+-------------------+------+--------------------+
 path| modificationTime|length| content|
+--------------------+-------------------+------+--------------------+
s3a://s3-jv-ocr/r...|2021-09-30 12:48:56| 5005|[FF D8 FF E0 00 1...|
s3a://s3-jv-ocr/r...|2021-09-30 12:48:56| 4984|[FF D8 FF E0 00 1...|
s3a://s3-jv-ocr/r...|2021-09-30 12:48:56| 4496|[FF D8 FF E0 00 1...|
s3a://s3-jv-ocr/r...|2021-09-30 12:48:55| 3928|[FF D8 FF E0 00 1...|
s3a://s3-jv-ocr/r...|2021-09-30 12:48:55| 3643|[FF D8 FF E0 00 1...|
s3a://s3-jv-ocr/r...|2021-09-30 12:48:54| 3382|[FF D8 FF E0 00 1...|
s3a://s3-jv-ocr/r...|2021-09-30 12:48:55| 2919|[FF D8 FF E0 00 1...|
s3a://s3-jv-ocr/r...|2021-09-30 12:48:55| 2875|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+

In [0]:
display(image_df)

path modificationTime length content s3a://s3-jv-ocr/reduced_dataset/Kiwi/174_100.jpg 2021-09-30T12:48:56.000+0000 5005 List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...) s3a://s3-jv-ocr/reduced_dataset/Kiwi/0_100.jpg 2021-09-30T12:48:56.000+0000 4984 List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...) s3a://s3-jv-ocr/reduced_dataset/Cherry_1/0_100.jpg 2021-09-30T12:48:56.000+0000 4496 List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...) s3a://s3-jv-ocr/reduced_dataset/Cherry_1/153_100.jpg 2021-09-30T12:48:55.000+0000 3928 List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAIAAAD/gAIDAAA5VUlEQVR42t2dB5Rb1dW2773nNnWNNCONRtKojHrX9N6rp7tjg40b3aYGTLMxzYEApiQEAsGmJIHQSSFAmpNQkgAGYwOhhFBD6C1gT/O/zzlXd+TxwAfk48ta/6y9hDyWsfXMu/d592li9n3Fr6kpGjO/yG9Nf01OTqp/ZGJiampiEmIfvGoS/vAE/Ac/m8K/AU8mJ8fxd+j3p8bVuOP2Wzeff+7Zmzaed+7ZF16w+YrLLx0fh1fuG5sYn6J/YAr+d8ovp+j/iLyAfuF/w9S+sT179+G/czL/n/T1vpj/LVgUEKWWe+XUBP4i3yHvBv7FE+N74V3Qx8mJsZ//7O6bbtx64w3btl5z1Q+u/N6WCzdvPnvjt45bu/KQJQtHh2oqyqPBQCwULPOURgJlqWR87sjomjVrNmzYsGXLlq1btz7yyCMqLwj1752cVNCMjY3Rv3pibPy/AOsLOe4nNOU7uZ/5lCKfydf+8dLtt/z4B1decf7ZGxfPGzlo/sj8gTlz5/QMd3VWxWOpQFmk1O21FRVqNRatbBR5syzCo04QCnQ6vSjarZaQ35eIRxsb6latPPTyyy+9++47qTaVHw/BBwFP8PM8mefr7r8JayYj8hz+lXvHxx566E9//OP2+++796c//tHV373s5OPXLRwaGO7paq+vqc+k6lKJmkQs4/cECgs9JpNNFG2ybBMlqyDYNRq7TmcRxWK9vlDWWmRNoVZnEiWjIAI1q8HgcZZUlGeGhwauvurKe+656/3333/22Wdfff218Umc81OKoKf/PUDyvwZr6sDYv37Bk48//nDLlotPO/WUw1Yu7+tsq07Fs+FgbTIOgOoT0azfE3HYEsV2r04u4TgXz/t0BrdG6xJll6yB8BqNDkly6fUuvQEgOnQ6qyTZtNoCQTAgziQJJq1kt5oziWhXZ+s1P7jqyiu/e+fdd7zy2stj+8Yn9k3ugYSfnKAFDn5sU//FNDwQFi1b8PXBe+///YUX//bs03/Y/rvjjjl6qK+7OpUIuUrCrpLaWLQpFa+PhBIlxT6D3m/QOlmmlGPLBCEkyz5e9AhySG/ySVo3Et28BAHsSkTJIYg2JDhlbYlWVwwqE0UTQjqWkRhGRozVqKsuzy5aOP/UU0+55dabn33umWdf/BsIaSyXlVRr/wVYB2IaJz9AJfvGJ3Y98eQ137/yrNNOO3jhwtpMKuoprQiHW8uzXVVVbelkeam7VCPbWMaJ2DKNHJSkkCT4GCYiSmVIhAiJGh8ScPBimSh7BNEryxAlvAjg7Igv5gVQnBUhC49sOg3wMggcIAOVNTbUHLP2iG9fuHnbDVvfePOf+Zj+O2k4i6bwA6mgk1Pjn+3Z/sAD3zpm7VBnVzYYbEgmG1Op5lSqNhBMFZd4JAlnHMeCoLw88nJcRBSigpCQ5TBCEV6M8HKIEyOCJszLQUH2c0KZIAE1F4dKeQH+rE+jcYqCjeGckmwXBEBmkyUjYiExNTxr1ErhkH/x4oWbNm286qqr3nrn7f98HPyasOgPB0YZ1d3QQQdjwr+eBGD33XX36iVLaxOp6kikLZXpzJS3JVNVHl/YWAAaKWURBDDyIRTghbAAdPg4L+RCohFDIgRhJ4Z4oQzx8Hr4U5CwGDSCQA4eFZOw8VyhwIHKjDynQzgriwsLurvawKbBcPnu2+/Avw30rrotdYikj19ylPyaysIWgHhCOk7nxDY5tWfPPTffcsjovPpEojGRbE9nO1PZllC0ssQd1Rv9UI9YhJWC+CDiAQHkXVSQEoIIkSSR4mUaCSRBUHAqLz+HeXkQ5ybh4jkQWjHCYUdcIc8XioJFEsyyABKzWUwV6RSMLf/655vEAlP145+3OjjC15fX3deBpf7f1b8JfmgTez77/f33b/3elctHR1sz2ZZksitb3pVItwUiNQ5nXGeESuRnUBnLBVkURgIECApnX45UiseRQUokkQCR4HHEEA8vDvEoiLgyHiTJenkcHsQAOBBaCccUQ8FCHEisSMC8jDwyCMgoy+l47MxT1//jheepGcY2lUiJqukrVTHm61kq9RE/mZjc88m/H96+/Tubzj507tze6mrA1BFLtoairWXBGntJ1mSOiloFEMtFWB7ePAQknSooiibNCRlOyAIsDj+HoN+Pk9cDLyhqgAx4QfgBGcf4BF7lZedYG8cWIQRVzMzzeo4FR2bR60Ff3z73nLvuvP35555V//H5P/Jv3DpQi/Duu+8+8fiOh7f/8dLN315z0NLOyqrOTLY/W9kbT3YGws0ud6XFmtRoAFOUQzEcHLzzpCBApEQRIi3gyPACRBZhUhUcjgwBpyJLkIpGkakqI7w4CkutYlRfUL8KNRK4f8hHk06ur6445ugjb7rphhdeeGFG/Z360o6C+XqyUvPx6V27t137w0svvOjYNYf11te3xBO96fLBdLajLNBe6m0sspUbjXFZBEEBpiTiU7xAHympjChBZAURopzHQUlBlLMChIoMMhQEmMiNA5gXp8CiJR8Hz0NgXgKCZCwQeeiZDLIg82yBWd/S0nTccetuuEHhRdNQrfffYBoqGp7at2vnU5ddfMkp647vbWyuj8VbovGueGpOLNntD3Q6PXUFloxWkxBRHKGkgDKSAJESEERa5CHKRQGiQhDKeb6S56t4qYpTgvICoUHg9CRVLAHjpoAjIqAQYgMcS4u9SxkiEQQt9jgZBd6qkTQ8o5cFHjEFFkN7Z9vmzZt//etf5/+8v0llUY+HZx5w67rz0cfPPOmUwbaOykCwI53piCa6wtHeULir1Ndss1fp9MAIIkXQ

In [0]:
# Data restructuring

image_df = image_df.withColumn('label', split(col('path'), '/').getItem(4))
image_df = image_df.select('path', 'content', 'label')
image_df.show()

+--------------------+--------------------+--------+
 path| content| label|
+--------------------+--------------------+--------+
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...| Kiwi|
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...| Kiwi|
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...|Cherry_1|
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...|Cherry_1|
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...| Banana|
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...| Banana|
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...|Eggplant|
s3a://s3-jv-ocr/r...|[FF D8 FF E0 00 1...|Eggplant|
+--------------------+--------------------+--------+

# Model

Downloading of the ResNet50 model file for featurization, and truncate the last layer(s).

In [0]:
# Deeplearning model instanciation

model = ResNet50(include_top=False)
model.summary()  # verify that the top layer is removed

Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
 8192/94765736 [..............................] - ETA: 1s 4751360/94765736 [>.............................] - ETA: 0s13205504/94765736 [===>..........................] - ETA: 0s22011904/94765736 [=====>........................] - ETA: 0s30744576/94765736 [========>.....................] - ETA: 0s38330368/94765736 [===========>..................] - ETA: 0s46661632/94765736 [=============>................] - ETA: 0s55214080/94765736 [================>.............] - ETA: 0s63176704/94765736 [==================>...........] - ETA: 0s71729152/94765736 [=====================>........] - ETA: 0s80191488/94765736 [========================>.....] - ETA: 0s88571904/94765736 [===========================>..] - ETA: 0s94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type) Output Shape Param # Connected to 
==================================================================================================
input_1 (InputLayer) [(None, None, None, 0 
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D) (None, None, None, 3 0 input_1[0][0] 
__________________________________________________________________________________________________
conv1_conv (Conv2D) (None, None, None, 6 9472 conv1_pad[0][0] 
__________________________________________________________________________________________________
conv1_bn (BatchNormalization) (None, None, None, 6 256 conv1_conv[0][0] 
__________________________________________________________________________________________________
conv1_relu (Activation) (None, None, None, 6 0 conv1_bn[0][0] 
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D) (None, None, None, 6 0 conv1_relu[0][0] 
__________________________________________________________________________________________________
pool1_pool (MaxPooling2D) (None, None, None, 6 0 pool1_pad[0][0] 
__________________________________________________________________________________________________
conv2_block1_1_conv (Conv2D) (None, None, None, 6 4160 pool1_pool[0][0] 
__________________________________________________________________________________________________
conv2_block1_1_bn (BatchNormali (None, None, None, 6 256 conv2_block1_1_conv[0][0] 
__________________________________________________________________________________________________
conv2_block1_1_relu (Activation (None, None, None, 6 0 conv2_block1_1_bn[0][0] 
__________________________________________________________________________________________________
conv2_block1_2_conv (Conv2D) (None, None, None, 6 36928 conv2_block1_1_relu[0][0] 
__________________________________________________________________________________________________
conv2_block1_2_bn (BatchNormali (None, None, None, 6 256 conv2_block1_2_conv[0][0] 
__________________________________________________________________________________________________
conv2_block1_2_relu (Activation (None, None, N

## Functions

In [0]:
bc_model_weights = spark.sparkContext.broadcast(model.get_weights())
def model_fn():
    """
    Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
    """
    model = ResNet50(weights=None, include_top=False) 
    model.set_weights(bc_model_weights.value)
    return model
  
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([50, 50])
    arr = img_to_array(img)
    return preprocess_input(arr)
  
def featurize_series(model, content_series):
    
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    
    input = np.stack(content_series.map(preprocess)) 
    preds = model.predict(input)
    output = [p.flatten() for p in preds]
    return pd.Series(output)
  
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                                is a pandas Series of image data.
    '''
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
 warnings.warn(

In [0]:
# Application of functions
features_df = image_df.select(col("path"), col("label"), featurize_udf("content").alias("features"))
features_df.printSchema()
features_df.show()

root
-- path: string (nullable = true)
-- label: string (nullable = true)
-- features: array (nullable = true)
 |-- element: float (containsNull = true)

+--------------------+--------+--------------------+
 path| label| features|
+--------------------+--------+--------------------+
s3a://s3-jv-ocr/r...| Kiwi|[0.0, 0.0, 0.0, 0...|
s3a://s3-jv-ocr/r...| Kiwi|[0.0, 0.0, 0.0, 0...|
s3a://s3-jv-ocr/r...|Cherry_1|[1.5135554, 0.0, ...|
s3a://s3-jv-ocr/r...|Cherry_1|[0.27284646, 0.0,...|
s3a://s3-jv-ocr/r...| Banana|[6.8176374, 0.0, ...|
s3a://s3-jv-ocr/r...| Banana|[0.0, 2.1828494, ...|
s3a://s3-jv-ocr/r...|Eggplant|[6.3627596, 0.0, ...|
s3a://s3-jv-ocr/r...|Eggplant|[12.273788, 0.241...|
+--------------------+--------+--------------------+

In [0]:
# Modification of the type of data

features_vector = udf(lambda l: Vectors.dense(l), VectorUDT())
features_df = features_df.select(col("path"),  col("label"), features_vector(features_df["features"]).alias("features"))

# Dimensions reduction with a PCA

In [0]:
# standard scaler before doing the PCA

scaler = StandardScaler(withMean=True, withStd=True,
                              inputCol='features',
                              outputCol='features_scaled')
scaler_fitted = scaler.fit(features_df)
features_scaled_df = scaler_fitted.transform(features_df)

In [0]:
# I choose a number of dimensions pretty low because the cluster is on a single node to reduce the fees

pca = PCA(k=16, inputCol="features_scaled", outputCol="features_pca")
model = pca.fit(features_scaled_df)
df_transformed = model.transform(features_scaled_df)

In [0]:
df_transformed.show()

+--------------------+--------+--------------------+--------------------+--------------------+
 path| label| features| features_scaled| features_pca|
+--------------------+--------+--------------------+--------------------+--------------------+
s3a://s3-jv-ocr/r...| Kiwi|[0.0,0.0,0.0,0.0,...|[-0.7424928712597...|[48.6361456135668...|
s3a://s3-jv-ocr/r...| Kiwi|[0.0,0.0,0.0,0.0,...|[-0.7424928712597...|[43.4183972039452...|
s3a://s3-jv-ocr/r...|Cherry_1|[1.51355540752410...|[-0.4124547305247...|[20.9121233015843...|
s3a://s3-jv-ocr/r...|Cherry_1|[0.27284646034240...|[-0.6829973694565...|[16.2893241496773...|
s3a://s3-jv-ocr/r...| Banana|[6.81763744354248...|[0.74412623424939...|[-63.319803123157...|
s3a://s3-jv-ocr/r...| Banana|[0.0,2.1828494071...|[-0.7424928712597...|[-49.043651966326...|
s3a://s3-jv-ocr/r...|Eggplant|[6.36275959014892...|[0.64493789912189...|[-13.697762151578...|
s3a://s3-jv-ocr/r...|Eggplant|[12.2737884521484...|[1.93386658038928...|[-3.1947730277129...|
+--------------------+--------+--------------------+--------------------+--------------------+

# Export the data scaled to S3

In [0]:
# Data selection and transformation to pandas

df_pandas = df_transformed.select('path', 'label', 'features_pca').toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field features_pca. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
# Export to S3 bucket

from io import StringIO

bucket = "s3-jv-ocr" 

csv_buffer = StringIO()
df_pandas.to_csv(csv_buffer)

s3_resource = boto3.resource('s3')

s3_resource.Object(bucket, 'df_reduced_pca_16.csv').put(Body=csv_buffer.getvalue())

Out[17]: {'ResponseMetadata': {'RequestId': '4FVJ9X4JSYJDKR2P',
 'HostId': 'faEpGvKYFAOpBffUXe26/5xHy1u9tgtPszvycvmckFk8Z6MBl/fSgpCqY2sbZKLsthAGwqunRn4=',
 'HTTPStatusCode': 200,
 'HTTPHeaders': {'x-amz-id-2': 'faEpGvKYFAOpBffUXe26/5xHy1u9tgtPszvycvmckFk8Z6MBl/fSgpCqY2sbZKLsthAGwqunRn4=',
 'x-amz-request-id': '4FVJ9X4JSYJDKR2P',
 'date': 'Tue, 19 Oct 2021 12:13:35 GMT',
 'etag': '"2a18f08a205cd29b8cca6e0f3e260766"',
 'server': 'AmazonS3',
 'content-length': '0'},
 'RetryAttempts': 1},
 'ETag': '"2a18f08a205cd29b8cca6e0f3e260766"'}